In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
pip install torch diffusers transformers accelerate -q
pip install langchain chromadb sentence-transformers PyPDF2 transformers accelerate bitsandbytes -q
pip install -U langchain-community -q
pip install pypdf -q
pip install -U bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 46.0 MB/s eta 0:00:00


In [ ]:
import os
import torch
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from diffusers import StableDiffusionPipeline
from PIL import Image

In [ ]:
# 1. Extract text from PDF
loader = PyPDFLoader("filePathOfTheTextbook")
pages = loader.load_and_split()

# 2. Split text into chunks with overlap
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
chunks = text_splitter.split_documents(pages)

# 3. Create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 4. Set up vector store
vectorstore = Chroma.from_documents(chunks, embeddings)

In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)
llm = HuggingFacePipeline(pipeline=pipe)

# Create retrieval QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
)

In [ ]:
def generate_image_prompt(answer):
    prompt = f"""
    Based on the following answer, create a concise and descriptive prompt for generating an image that visualizes or summarizes the key points:

    Answer: {answer}

    Image prompt:
    """
    result = pipe(prompt, max_new_tokens=50, do_sample=True, top_k=50, top_p=0.95)
    image_prompt = result[0]['generated_text'].split("Image prompt:")[-1].strip()
    return image_prompt

def generate_image(prompt, output_path):
    try:
        # Try loading the first choice of model
        model_id = "CompVis/stable-diffusion-v1-4"
        pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    except Exception as e:
        print(f"Error loading first choice model: {e}")
        try:
            # Fallback to a different model
            model_id = "stabilityai/stable-diffusion-2-base"
            pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
        except Exception as e:
            print(f"Error loading fallback model: {e}")
            return None

    # Move the model to GPU if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    pipe = pipe.to(device)

    try:
        # Generate the image
        image = pipe(prompt).images[0]

        # Save the image
        image.save(output_path)
        print(f"Image saved to {output_path}")

        return image
    except Exception as e:
        print(f"Error generating image: {e}")
        return None

def query_rag_with_image(question):
    # Get answer from RAG system
    result = qa_chain({"query": question})
    answer = result["result"]
    sources = result["source_documents"]

    # Generate image prompt
    image_prompt = generate_image_prompt(answer)

    # Generate and save image
    output_path = f"generated_image_{question[:20]}.png".replace(" ", "_")
    image = generate_image(image_prompt, output_path)

    return answer, sources, image, image_prompt, output_path if image else None

# Example usage
question = "What is neural network?"
answer, sources, image, image_prompt, image_path = query_rag_with_image(question)

print(f"Answer: {answer}")
print(f"\nImage Prompt: {image_prompt}")
if image_path:
    print(f"\nImage saved to: {image_path}")
else:
    print("\nImage generation failed.")
print("\nSources:")
for i, doc in enumerate(sources):
    print(f"Source {i+1}: {doc.page_content[:100]}...")

# You can now open the image using a standard image viewer or display it in a notebook, if it was generated successfully
# if image_path:
#     Image.open(image_path).show()

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Image saved to generated_image_What_is_the_8th_chap.png
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Watch us on Y ouTube: http://www.youtube.com/oreillymedia
Acknowledgments
First, I would like to thank Mike Loukides for accepting my proposal for this book
(and for insisting that I pare it down to a reasonable size). It would have been very
easy for him to say, “Who’s this person who keeps emailing me sample chapters, and
Preface | xv

Watch us on Y ouTube: http://www.youtube.com/oreillymedia
Acknowledgments
First, I would like to thank Mike Loukides for accepting my proposal for this book
(and for insisting that I pare it down to a reasonable size). It would have been very
easy for him to say, “Who’s this person who keeps emailing me sample chapters, and
Preface | xv

at which point we can see how the model assigns topics to each user’s interests:
for documen

In [ ]:
question = "What is neural network?"
answer, sources, image, image_prompt, image_path = query_rag_with_image(question)

print(f"Answer: {answer}")
print(f"\nImage Prompt: {image_prompt}")
if image_path:
    print(f"\nImage saved to: {image_path}")
else:
    print("\nImage generation failed.")
print("\nSources:")
for i, doc in enumerate(sources):
    print(f"Source {i+1}: {doc.page_content[:100]}...")

# You can now open the image using a standard image viewer or display it in a notebook, if it was generated successfully
# if image_path:
#     Image.open(image_path).show()

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Image saved to generated_image_What_is_neural_netwo.png
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

CHAPTER 18
Neural Networks
I like nonsense; it wakes up the brain cells.
—Dr. Seuss
An artificial  neural network  (or neural network for short) is a predictive model moti‐
vated by the way the brain operates. Think of the brain as a collection of neurons
wired together. Each neuron looks at the outputs of the other neurons that feed into
it, does a calculation, and then either fires (if the calculation exceeds some thresh‐
hold) or doesn’t (if it doesn’t).
Accordingly, artificial neural networks consist of artificial neurons, which perform
similar calculations over their inputs. Neural networks can solve a wide variety of
problems like handwriting recognition and face detection, and they are used heavily
in deep learning, one of the trendiest subfields of data 